
# Regression model
# Relative location of CT slices on axial axis

 The data are available at:

 https://archive.ics.uci.edu/dataset/206/relative+location+of+ct+slices+on+axial+axis

The dataset consists of 384 features extracted from CT images. The class variable is numeric and denotes the relative location of the CT slice on the axial axis of the human body.

The data was retrieved from a set of 53500 CT images from 74 different patients (43 male, 31 female).

To exstract the data use the panda routines


In [32]:
import zipfile
import pandas as pd
 
# read the dataset using the compression zip
df = pd.read_csv('https://archive.ics.uci.edu/static/public/206/relative+location+of+ct+slices+on+axial+axis.zip',compression='zip')
 
# display dataset
print(df.head())

   patientId  value0  value1  value2  value3  value4  value5  value6  value7  \
0          0     0.0     0.0     0.0     0.0     0.0     0.0   -0.25   -0.25   
1          0     0.0     0.0     0.0     0.0     0.0     0.0   -0.25   -0.25   
2          0     0.0     0.0     0.0     0.0     0.0     0.0   -0.25   -0.25   
3          0     0.0     0.0     0.0     0.0     0.0     0.0   -0.25   -0.25   
4          0     0.0     0.0     0.0     0.0     0.0     0.0   -0.25   -0.25   

   value8  ...  value375  value376  value377  value378  value379  value380  \
0   -0.25  ...     -0.25  0.980381       0.0       0.0       0.0       0.0   
1   -0.25  ...     -0.25  0.977008       0.0       0.0       0.0       0.0   
2   -0.25  ...     -0.25  0.977008       0.0       0.0       0.0       0.0   
3   -0.25  ...     -0.25  0.977008       0.0       0.0       0.0       0.0   
4   -0.25  ...     -0.25  0.976833       0.0       0.0       0.0       0.0   

   value381  value382  value383  reference  
0    

We transform the data to a matrix of shape 53500 x 386

In [33]:
Aall=df.to_numpy()
print(Aall.shape)

(53500, 386)


We add a column of all 1 and we organize the input data by dividing in test set and training set

In [34]:
from sklearn.model_selection import train_test_split
import numpy as np
#Add a column of ones at the beginning of the data matrix
Aall = np.column_stack([np.ones(Aall.shape[0]), Aall])
X = Aall
X=np.delete(X,386,1)
y = Aall[:,386]
print(X[:,0])

[1. 1. 1. ... 1. 1. 1.]


In [35]:
import numpy as np
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    train_size = .9,
    test_size = .1,
    random_state = 5,
    shuffle = True
)


In [36]:
print(X_train.shape)
print(X_test.shape)

(48150, 386)
(5350, 386)


Use the prepared data to solve the regression model with all the studied techniques.
Can we use the normal equation and the QR factorization? If the answer is positive compare the condition numbers of the QR methods and the normal equations. What are the results?


Use the funcation scipy.linalg.lstsq and check if all the lapack drivers works.
Compare the results changing the initial value cond. The results are the same? What about the execution time?

Analyze the singular values and check if it is possible to use a principal component regression procedure. Compute the solution using the singular value decomposition. 
Can you observe a relation in the chosen singular value and the value of cond of the routine lstsq?

Perform the same analysis by preprocessing the data in order to have data from a normal distribution with mean zero  and compute the singular value decomposition on this matrix.

Check the performance of the method by computing the least square residual for the training set and the testset. The minimum and the maximum values of the predicted error for both, the training set and the testset.

Compute the multiple R-squared: R2_train = 1 - sum( (y - yest)**2)/sum( (y-mean(y))**2 where y are the value to predict and yest are the estimated values for the training set.
Compute the value R2_test for the testset.

A value of R2 near one means that the constructed model is good.

Change the size of the training set and the testing set to 0.7% and 0.3% and repeat the previous steps.

Comment the obtained results.


<h1> Normal Equation </h1>
Normal equation is a method to solve the linear regression problem. It is based on the following formula: 

$$\theta = (X^T X)^{-1} X^T y$$

where $\theta$ is the vector of parameters, $X$ is the matrix of input data, and $y$ is the vector of output data. It is important to note that the matrix $X$ must be full rank, otherwise the inverse of $X^T X$ does not exist, so the normal equation cannot be used.


Since the matrix $X$ is not full rank, we cannot use the normal equation to solve the problem. The rank of the matrix $X^T X$ is 375, which is lower than p=min(n,m)=386. The condition number of the matrix $X^T X$ is very high, which means that the matrix is ill-conditioned. The condition number of a matrix is computer as follows: 

$$
\|A\|_2 \cdot \|A^{-1}\|_2
$$
We can notice that, if we try, the residual error is very high 



In [37]:
print("Rank of train data: ",np.linalg.matrix_rank(X_train.T @ X_train))
print("Shape of train data: ",(X_train.T @ X_train).shape)
theta = np.linalg.solve(X_train.T @ X_train, X_train.T @ y_train)
print("Residuals: ",np.linalg.norm(X_train @ theta - y_train))
print("Condition number of normal equation matrix: ",np.linalg.cond(X_train.T @ X_train))

Rank of train data:  375
Shape of train data:  (386, 386)
Residuals:  5.583855310705728e+19
Condition number of normal equation matrix:  1.8704153273192688e+22


<QR factorization>
<!-- Write a description of the problem here, with used formulas IN MARKDOWN -->
The QR factorization is a method to solve the linear regression problem. It is based on the following formula:

$$
X = QR
$$

where $X$ is the matrix of input data, $Q$ is an orthogonal matrix, and $R$ is an upper triangular matrix. The solution of the linear regression problem is given by:

$$
\theta = R^{-1} Q^T y
$$

Since $X$ is not full rank, we cannot use the QR factorization to solve the problem. We need to use the Pivot QR factorization, which is based on the following formula:

$$
X P = QR
$$

where $P$ is a permutation matrix. The solution of the linear regression problem is given by:

$$
\theta = R^{-1} Q^T P^T y
$$

